Mini Projet
Le but de cet exercice est d'extraire un sous-ensemble de données à partir d'une base de données
plus grande.
Supposons que vous ayez besoin de réaliser une application qui prend en entrée, des noms
de lieux du Burkina Faso, avec des informations telles que la latitude/longitude.
Pour ce faire, nous décidons d'extraire ces informations à partir du server de référencement géoname
(http://www.geonames.org/). Vous allez procéder de la façon suivante :
1 - Exporter la base de données qui recense les informations sur le Burkina Faso (https://download.geonames.org/export/dump/).
    o	Pour ce faire, reférez-vous au Readme, décrit à la fin de la page pour identifier le code iso correspondant au Burkina Faso
2 - Télécharger le fichier zip correspondant
3 - Appliquer les opérations de prétraitement et filtres nécessaires à ce fichier, pour ne garder que les colonnes correspondantes
    o	Identifiants, Noms de lieux, latitudes, longitudes
    o	Renommez les avec les noms suivants : 'ID', 'location_name', 'lat', 'long'
    o	Sauvegarder ces données dans un fichier CSV, nommez-le burkina_location.csv
4 - Opérations sur le fichier CSV burkina_location.csv.
    o	extraire les données contenant le nom 'gounghin', enregistrez-le sous le fichier gounghin.csv
    o	extraire la sous-partie de la base de données (fichier burkina_location.csv), dont les noms les premières lettres des noms de lieux sont compris entre 'A' et 'P' (ordre alphabétique)
    o	Identifiez respectivement, la latitude, la longitude minimale et les noms de lieux correspondants
    o	Quels sont les lieux dont les coordonnées sont comprises entre (lat >= 11 et lon <= 0.5)
5 - Sorties Excel
À partir des extractions de l'étape 4 :
•	Créer un fichier Excel et nommer le : mini_projet
•	Créer une feuille dans ce fichier, du nom gounghin et enregistrer les données contenant le nom 'gounghin' obtenues dans 4.1
•	Créer une second feuille dans ce même fichier, du nom A_to_P et enregistrer les données de 4.2
•	Chacun devrait rendre :
1- un fichier (Jupyter ou Google Colab) à partir duquel, le code entré et la sortie seront bien visibles.
2- Un repertoire github (lien d'accès) associé au nom (mini_projet_data_analysis), dans lequel il mettra les données 


## 1) Imports & fonctions utilitaires

In [1]:

import os
import io
import zipfile
import time
from pathlib import Path

import pandas as pd
# Utiliser requests si disponible, sinon urllib
try:
    import requests
    HAS_REQUESTS = True
except Exception:
    import urllib.request
    HAS_REQUESTS = False

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

BF_URL = "https://download.geonames.org/export/dump/BF.zip"
BF_ZIP = DATA_DIR / "BF.zip"
BF_TXT = DATA_DIR / "BF.txt"

def download_file(url, dest_path, max_retries=3, timeout=60):
    """
    Télécharge un fichier avec retries. Utilise requests si dispo, sinon urllib.
    """
    for attempt in range(1, max_retries+1):
        try:
            if HAS_REQUESTS:
                with requests.get(url, stream=True, timeout=timeout) as r:
                    r.raise_for_status()
                    with open(dest_path, "wb") as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            if chunk:
                                f.write(chunk)
            else:
                with urllib.request.urlopen(url, timeout=timeout) as r:
                    with open(dest_path, "wb") as f:
                        f.write(r.read())
            return True
        except Exception as e:
            print(f"[Retry {attempt}/{max_retries}] Erreur de téléchargement: {e}")
            time.sleep(2 * attempt)
    return False

def extract_zip(zip_path, to_dir):
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extractall(to_dir)
    print(f"Extraits: {list(Path(to_dir).iterdir())}")


## 2) Télécharger & extraire le dump GeoNames pour le Burkina Faso (`BF`)

In [4]:

if not BF_ZIP.exists():
    ok = download_file(BF_URL, BF_ZIP)
    if not ok:
        raise RuntimeError("Échec du téléchargement de BF.zip. Vérifiez la connexion Internet et ré-exécutez la cellule.")
else:
    print("BF.zip déjà présent — on réutilise le fichier local.")

extract_zip(BF_ZIP, DATA_DIR)
if not BF_TXT.exists():
    # Dans certains dumps, le nom peut être 'BF.txt' (standard). Vérifions tout de même.
    raise FileNotFoundError("Le fichier BF.txt n'a pas été trouvé après extraction.")
print("Prêt à lire :", BF_TXT)


BF.zip déjà présent — on réutilise le fichier local.
Extraits: [WindowsPath('data/BF.txt'), WindowsPath('data/BF.zip'), WindowsPath('data/readme.txt')]
Prêt à lire : data\BF.txt


## 3) Charger le fichier et filtrer/renommer les colonnes utiles

In [5]:

# Schéma attendu (GeoNames): https://download.geonames.org/export/dump/readme.txt
cols = [
    'geonameid', 'name', 'asciiname', 'alternatenames',
    'latitude', 'longitude', 'feature class', 'feature code',
    'country code', 'cc2', 'admin1 code', 'admin2 code',
    'admin3 code', 'admin4 code', 'population', 'elevation',
    'dem', 'timezone', 'modification date'
]

# Charger en chaîne pour éviter les surprises, conversion plus tard
df = pd.read_csv(BF_TXT, sep="\t", names=cols, dtype=str, quoting=3, header=None, low_memory=False)

# Garder colonnes utiles et renommer
df_filtered = df[['geonameid', 'name', 'latitude', 'longitude']].copy()
df_filtered.columns = ['ID', 'location_name', 'lat', 'long']

# Nettoyage basique
df_filtered['location_name'] = df_filtered['location_name'].astype(str).str.strip()

# Convertir lat/long en float de manière robuste
df_filtered['lat'] = pd.to_numeric(df_filtered['lat'], errors='coerce')
df_filtered['long'] = pd.to_numeric(df_filtered['long'], errors='coerce')

# Supprimer les lignes sans coordonnées valides
df_filtered = df_filtered.dropna(subset=['lat', 'long']).reset_index(drop=True)

# Sauvegarder CSV principal
df_filtered.to_csv("burkina_location.csv", index=False, encoding="utf-8")
print("✅ Sauvegardé 'burkina_location.csv'")
df_filtered.head()


✅ Sauvegardé 'burkina_location.csv'


,ID,location_name,lat,long
0,2282318,Pouéné,9.72908,-2.78660
1,2285251,Léraba Occidentale,10.28333,-5.11667
2,2287216,Kéléouoro,9.80748,-4.05023
3,2294066,White Volta,8.70194,-0.99056
4,2298457,Issana Bouga,10.91667,-1.18333


## 4) Opérations de requêtage sur `burkina_location.csv`

In [8]:

df_base = pd.read_csv("burkina_location.csv")

# 4.1 Filtrer noms contenant 'gounghin' (insensible à la casse)
gounghin_df = df_base[df_base['location_name'].str.contains("gounghin", case=False, na=False)]
gounghin_df.to_csv("gounghin.csv", index=False, encoding="utf-8")
print(f"✅ 'gounghin.csv' enregistré ({len(gounghin_df)} lignes)")

# 4.2 Extraire les noms dont la 1ère lettre est entre 'A' et 'P' (insensible à la casse)
first_letter = df_base['location_name'].astype(str).str[0].str.upper()
a_to_p_mask = first_letter.between('A', 'P', inclusive='both')
a_to_p_df = df_base[a_to_p_mask].copy()
print(f"Sous-ensemble A->P : {len(a_to_p_df)} lignes")

# 4.3 Latitude/Longitude minimales + noms correspondants
min_lat = df_base['lat'].min()
min_lat_names = df_base.loc[df_base['lat'] == min_lat, 'location_name'].unique().tolist()

min_long = df_base['long'].min()
min_long_names = df_base.loc[df_base['long'] == min_long, 'location_name'].unique().tolist()

print(f"Latitude minimale: {min_lat}  | Lieux: {min_lat_names}")
print(f"Longitude minimale: {min_long} | Lieux: {min_long_names}")

# 4.4 Lieux avec lat >= 11 et lon <= 0.5
subset_coords_df = df_base[(df_base['lat'] >= 11) & (df_base['long'] <= 0.5)].copy()
print(f"Lignes (lat >= 11 & lon <= 0.5): {len(subset_coords_df)}")

# Aperçus
display(gounghin_df.head())
display(a_to_p_df.head())
display(subset_coords_df.head())


✅ 'gounghin.csv' enregistré (10 lignes)
Sous-ensemble A->P : 8306 lignes
Latitude minimale: 5.21609  | Lieux: ['Komoé']
Longitude minimale: -5.65968 | Lieux: ['Banifing']
Lignes (lat >= 11 & lon <= 0.5): 9466


,ID,location_name,lat,long
153,2353306,Gounghin,12.06677,-1.42134
7269,2360473,Gounghin,12.62488,-1.36398
10260,2570204,Gounghin,12.31436,-1.37900
10746,10342749,Gounghin,12.06667,-0.15000
10759,10629032,BICIAB // Gounghin,12.35921,-1.54273


,ID,location_name,lat,long
0,2282318,Pouéné,9.72908,-2.78660
1,2285251,Léraba Occidentale,10.28333,-5.11667
2,2287216,Kéléouoro,9.80748,-4.05023
4,2298457,Issana Bouga,10.91667,-1.18333
137,2353290,Forêt Classée de Ziga,12.47106,-1.08644


,ID,location_name,lat,long
5,2353158,Zyonguen,12.36667,-0.45000
6,2353159,Zyiliwèlè,12.38333,-2.73333
7,2353160,Zyanko,12.78333,-0.41667
8,2353161,Zouta,13.14908,-1.28197
9,2353162,Zourtenga,12.95741,-1.28745


## 5) Générer le fichier Excel `mini_projet.xlsx`

In [9]:
pip install xlsxwriter


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:

with pd.ExcelWriter("mini_projet.xlsx", engine="xlsxwriter") as writer:
    gounghin_df.to_excel(writer, sheet_name="gounghin", index=False)
    a_to_p_df.to_excel(writer, sheet_name="A_to_P", index=False)
print("✅ Fichier Excel 'mini_projet.xlsx' créé avec feuilles 'gounghin' et 'A_to_P'")


✅ Fichier Excel 'mini_projet.xlsx' créé avec feuilles 'gounghin' et 'A_to_P'


## 6) Résumé des sorties produites

In [16]:

from pathlib import Path
outputs = {
    "CSV principal": str(Path("burkina_location.csv").resolve()),
    "Filtre 'gounghin'": str(Path("gounghin.csv").resolve()),
    "Excel": str(Path("mini_projet.xlsx").resolve())
}
pd.DataFrame.from_dict(outputs, orient="index", columns=["Chemin"]).T


,CSV principal,Filtre 'gounghin',Excel
Chemin,C:\Users\ouedr\Desktop\Data viz\burkina_locati...,C:\Users\ouedr\Desktop\Data viz\gounghin.csv,C:\Users\ouedr\Desktop\Data viz\mini_projet.xlsx
